# GIS Example File
This file serves as an example for the GIS handler used to integrate GIS data with Python models. The handler can be loaded from the cell below, for use in a notebook, or from the `gis_handler.py` file which is identical. The examples in this file demonstrate possible ways to use the handler in modeling.

## Setup

In [1]:
# import packages required for the examples
import matplotlib.pyplot as plt
import rasterio.plot

In [2]:
# specify plot settings such as figure size
plt.rcParams["figure.figsize"] = [10,8]
%config InlineBackend.figure_format = 'retina'

## Handler

In [3]:
# Python GIS Handler
# SEA Lab at Cornell University, last updated: 4/24/23

# import necessary packages, namely geopandas and rasterio
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import warnings
from shapely.errors import ShapelyDeprecationWarning

# ignore shapely 1.8 deprecation warnings
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
precision = 5 # precision of input coordinates to prevent duplicate calls, 5dec~1.1m

class GISHandler:
    """A class to handle GIS data and optimizer points."""
        
    def __init__(self, conditions, conflicts, scope):
        """Initializes handler by creating a GeoDataFrame to store point measurement data and a dictionary of loaded raster files."""
        self.conditions = {}
        self.conflicts = {}
        self.scope = gpd.read_file(scope)
        # create initial geodataframe with anticipated fields
        self.points = gpd.GeoDataFrame(columns=['x', 'y', 'geometry', 'result', 'ok-conditions', 'ok-scope', 'ok-conflicts'], geometry='geometry')
    
        # check if raster already loaded as a condition, otherwise read source
        for key, src in conditions.items():
            if key in self.conditions:
                print('raster {} already loaded!'.format(key))
            else:
                self.conditions[key] = rasterio.open(src)
        
        # check if vector already loaded as a conflict, otherwise read source
        for key, src in conflicts.items():
            if key in self.conflicts:
                print('vector {} already loaded!'.format(key))
            else:
                self.conflicts[key] = gpd.read_file(src)
                
        # calculate boundaries of loaded files
        self.extent = self.extent()
    
    def query(self, x, y):
        """Gets condition data for a specified geography location (lon/lat), stores it in the GeoDataFrame, and returns the row."""
        x, y = self.coordinate(x, y)
                                
        # check if point has already been queried
        if not self.points.loc[(self.points.x==x) & (self.points.y==y)].empty:
            #print('point exists, returning original data')
            return self.points.loc[(self.points.x==x) & (self.points.y==y)]
        
        point = Point(x, y)
        conditions = {'x': x, 'y': y, 'geometry': point, 'ok-conditions': True, 'ok-scope': False, 'ok-conflicts': True}
        
        # check if point is offshore in desired scope
        for polygon in self.scope['geometry']:
            if point.intersects(polygon):
                conditions['ok-scope'] = True
        
        # check if point intersects with any of the conflicts loaded as vectors
        for key, vector in self.conflicts.items():
            for polygon in vector['geometry']:
                if point.intersects(polygon):
                    conditions['ok-conflicts'] = False
        
        # check if point exists for all condition datasets, and pull condition data for point
        for key, raster in self.conditions.items():
            index = raster.index(x, y)
            try:
                conditions[key] = raster.read(1)[index] # yes, by default this only reads the first band, but this is probably okay
                if conditions[key] == 0: # yes, this assumes that zero is not a valid value. this is true for our current rasters, but isn't necessarily correct
                    conditions['ok-conditions'] = False
            except IndexError as error:
                print('failed to read {} raster: {}'.format(key, error))
                conditions['ok-conditions'] = False
        
        self.points = self.points.append(conditions, ignore_index=True)
        return self.points.iloc[-1:]
    
    def record(self, x, y, value):
        """Records a computed value from the optimizer to a geographic point, returns row recorded to."""
        x, y = self.coordinate(x, y)
        
        if not self.points.loc[(self.points.x==x) & (self.points.y==y)].empty:
            self.points.loc[(self.points.x==x) & (self.points.y==y), 'result'] = value
            return self.points.loc[(self.points.x==x) & (self.points.y==y)]
        
        conditions = {'x': x, 'y': y, 'geometry': Point(x, y), 'result': value}
        self.points = self.points.append(conditions, ignore_index=True)
        return self.points.iloc[-1:]
    
    def query_grid(self, x_min, x_max, y_min, y_max, xy_delta):
        """Iteratively builds a grid of points with desired bounds and resolution for brute-force analysis."""
        self.grid = gpd.GeoDataFrame(columns=['x', 'y', 'geometry', 'result', 'ok-conditions', 'ok-scope', 'ok-conflicts'], geometry='geometry')
        
        x = x_min
        y = y_min
        while x <= x_max:
            while y <= y_max:
                self.query(x, y)
                y += xy_delta
            y=y_min
            x += xy_delta
    
    def save(self, name):
        """Saves all loaded points and values into GIS format."""
        self.points.to_file(name, driver='GeoJSON')
        
    def load(self, name):
        """Loads previously saved points from GIS file."""
        self.points = gpd.read_file(name)
        
    def coordinate(self, x, y):
        """Rounds coordinates to given precision to prevent uneccessary duplication. In the future could handle projections."""
        return round(x, precision), round(y, precision)
    
    def extent(self):
        """Calculates largest rectangular extent that includes data from all loaded rasters."""
        
        extent = [-180, 180, -90, 90] # format: [W, E, S, N]
        for src in self.conditions.values():
            extent = [max(extent[0], src.bounds[0]), min(extent[1], src.bounds[2]), max(extent[2], src.bounds[1]), min(extent[3], src.bounds[3])]
            
        return extent

## Example Usage

### Load conditions (rasters), conflicts (vectors), and scope (vector)

In [4]:
# specify raster files with conditions
conditions = {'current [m/s]': '../data/Surface Currents m-s (NODP 2016).tif',
              'oxygen [mg/l]': '../data/Surface Oxygen mg-l (NCEI 2019).tif',
              'salinity [PSU]': '../data/Surface Salinity PSU (NCEI 2019).tif',
              'temperature [°C]': '../data/Surface Temperature C (NODP 2016).tif',
              'period [s]': '../data/Wave Energy Period s (NREL 2011).tif',
              'height [m]': '../data/Significant Wave Height m (NREL 2011).tif',
              'bathymetry [m]': '../data/Bathymetry Downsampled m (NGDC 1990).tif',
              'distance to port [m]': '../data/Distance to Port m (OCM 2019).tif'}

# specify vector files with conflicts
# high fishing is above average, very high is more than one standard deviation above average
conflicts = {'very high fishing traffic': '../data/Very High Fishing Vessel Traffic (NODP 2022).geojson',
#            'high fishing traffic': '../data/High Fishing Vessel Traffic.geojson',
             'marine protected areas': '../data/Marine Protected Areas (NMPAC 2020).geojson',
             'danger zones': '../data/Danger Zones and Restricted Areas (OCM 2022).geojson',
             'submarine': '../data/Submarine Transit Lanes (NODP 2016).geojson',
             'torpex': '../data/Cape Cod TORPEX (NODP 2016).geojson',
             'block island': '../data/Block Island Renewable Energy Zone (NODP 2010).geojson',
             'ma wind': '../data/MA Wind Energy Areas (NODP 2015).geojson',
             'wind lease': '../data/Potential Wind Lease Areas (BOEM 2023).geojson',
             'wind planning': '../data/Wind Planning Areas (BOEM 2023).geojson',
             'shipping': '../data/Shipping Lanes (OCS 2015).geojson'}

# specify vector file with desired scope
waters = "../data/Northeast State and Federal Waters (OCM 2018).geojson"

# create handler object given specified files to load
handler = GISHandler(conditions, conflicts, waters)

### Basic Operations

#### Print Extent

In [5]:
print('Extent: W', handler.extent[0], 'E', handler.extent[1], 'S', handler.extent[2], 'N', handler.extent[3])

Extent: W -75.766660333 E -65.701105095 S 38.500067 N 45.166667


#### Query Point in Format (longitude, latitude) to Get Data

In [6]:
handler.query(-70.154, 42.203)

AttributeError: 'GeoDataFrame' object has no attribute 'append'

In [ ]:
handler.query(-72, 41.2)

#### Record Result of Model into GeoDataFrame

In [ ]:
# example result from model: 10
handler.record(-72, 41.2, 10)

#### Generate a Grid of Points

In [ ]:
%%time
handler.query_grid(-72, -68, 40, 44, 1)

#### Save Results to File

In [ ]:
handler.save('saved.geojson')

### Show Results with Tables and Maps

In [ ]:
handler.points

In [ ]:
fig, ax = plt.subplots(2, 2)
rasterio.plot.show(handler.conditions['height [m]'], ax=ax[0,0], title='Wave Height [m]')
rasterio.plot.show(handler.conditions['distance to port [m]'], ax=ax[0,1], title='Distance to Port [m]')
rasterio.plot.show(handler.conditions['bathymetry [m]'], ax=ax[1,0], title='Bathymetry [m]')
rasterio.plot.show(handler.conditions['temperature [°C]'], ax=ax[1,1], title='Temperature [°C]')

for i in ax:
    for j in i:
        handler.points.plot(ax=j, color='red')

In [ ]:
%%time
fig, ax = plt.subplots()
handler.scope.plot(edgecolor='black', color='white', ax=ax)
for gdf in handler.conflicts.values():
    gdf.plot(ax=ax)
handler.points.plot(ax=ax, color='red')